In [1]:
%%capture
! pip install -q --upgrade pypdf llama-index llama-index-llms-gemini llama-index-embeddings-gemini

In [2]:
## get the data
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-06-29 06:59:30--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  52.1MB/s    in 0.3s    

2024-06-29 06:59:31 (52.1 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [3]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
import torch
import nest_asyncio
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.selectors import LLMMultiSelector
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import StorageContext

nest_asyncio.apply()

In [5]:
##Load the data
documents = SimpleDirectoryReader(input_files=['metagpt.pdf']).load_data()

##split the document
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [7]:
gemini_embedding_model = GeminiEmbedding(model_name="models/embedding-001")
llm = Gemini(model_name="models/gemini-1.5-flash-latest")

In [8]:
# Set Global settings
Settings.llm = llm
Settings.embed_model = gemini_embedding_model

In [9]:
# llm  = LlamaCPP(
#    model_url='https://huggingface.co/bartowski/Llama-3-8B-Instruct-Gradient-1048k-GGUF/resolve/main/Llama-3-8B-Instruct-Gradient-1048k-Q5_K_S.gguf',
#    model_path=None,
#    temperature=0.1,
#    max_new_tokens=256,
#    context_window=3900,
#    generate_kwargs={},
#    #model_kwargs={"n_gpu_layers":-1},
#    verbose=True
# )
# Settings.llm = llm
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [10]:
##define summary and vector index on the same data
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

##Define Query Engines and Set Metadata
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)
vector_query_engine = vector_index.as_query_engine()

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description = (
        "Useful for Summarization questions related to MetaGPT"
    )
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper"
    )
)

In [11]:
query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    #prompt_template="{query_str}\n\nPlease provide the answer in the following JSON format:\n",
    query_engine_tools=[summary_tool,vector_tool],
    verbose=True
)

response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The question asks for a summary of the document, which aligns with the purpose of summarization..
This paper introduces MetaGPT, a meta-programming framework that leverages SOPs to enhance the problem-solving capabilities of multi-agent systems based on Large Language Models (LLMs). MetaGPT models a group of agents as a simulated software company, analogous to simulated towns and the Minecraft Sandbox in V oyager. MetaGPT leverages role specialization, workflow management, and efficient sharing mechanisms such as message pools and subscriptions, rendering it a flexible and portable platform for autonomous agents and multi-agent frameworks. It uses an executable feedback mechanism to enhance code generation quality during runtime. In extensive experiments, MetaGPT achieves state-of-the-art performance on multiple benchmarks. The successful integration of human-like SOPs inspires future research on human-inspired techniques for artificial multi-agent systems. 



In [12]:
response = query_engine.query("How do agents share information with other agents?")
print(str(response))

Selecting query engine 1: This choice is relevant because it focuses on retrieving specific context from the MetaGPT paper, which is likely to contain information about how agents share information..
The provided context does not contain information about how agents share information with other agents. 

